In [1]:
!pip install nltk transformers torch flask

In [ ]:
pip uninstall torch

In [ ]:
import nltk
import sqlite3
import torch
from transformers import pipeline, Conversation
from flask import Flask, request, jsonify
nltk.download('punkt')

In [ ]:

chatbot = pipeline("conversational", model="microsoft/DialoGPT-medium")

In [ ]:

conn = sqlite3.connect("chat_logs.db")
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS interactions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_query TEXT,
    bot_response TEXT
)''')
conn.commit()

In [ ]:

def get_response(user_input):
    conv = Conversation(user_input)
    response = chatbot(conv)
    bot_reply = response.generated_responses[-1]

    # Log interaction
    cursor.execute("INSERT INTO interactions (user_query, bot_response) VALUES (?, ?)", (user_input, bot_reply))
    conn.commit()

    return bot_reply

In [ ]:

user_input = "What are your working hours?"
response = get_response(user_input)
print("Bot:", response)

In [ ]:

app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get("message")
    response = get_response(user_input)
    return jsonify({"response": response})

if __name__ == '__main__':
    app.run(debug=True)